In [ ]:
%matplotlib inline
import numpy as np 
import matplotlib.pyplot as plt
import statistics as stat
import time

from matplotlib.backends.backend_pdf import PdfPages
from sympy.interactive import printing 
printing.init_printing(use_latex = (True))

# Model

In [ ]:
class MyError(Exception):
    pass

def Model(thick, conc, beam_diam, P_in, I0, n, dt, tf, wl, nw, filename):

    A_Circle = np.pi * (beam_diam/2)**2
    if P_in != 0: I0 = P_in/A_Circle 
    else: I0==I0
    k_F = 4.2 * 10**7 
    wlim = wl*k_F

    #### Fixed constants ####
    
    sigma = 2 * 10**(-17) * 10**(-4) 
    N_Avo = 6.02 * 10**23
    N0 = ((1.24*N_Avo*10**(6)*conc)/230)/100
    Px, Py, Pz = 0.76,0.16,0.08
    k_ISC = 6.9 * 10**7 
    k_Tx,k_Ty,k_Tz  = 2.8 * 10**4,0.6 * 10**4,0.2 * 10**4 
    k_31 = stat.mean([k_Tx*Px,k_Ty*Py,k_Tz*Pz]) 
    h, c, miu,a = 6.63 * 10**(-34),3 * 10**8,0.71 * 3.3356 * 10**(-30),1.6150
    h_bar = h/(2*np.pi)
    t_F = 1/k_F
    C1, C3 = (t_F/2)*((miu**2)*a)/(h_bar**2),(sigma*t_F)/np.pi
    dw = np.linspace(-wlim, wlim, nw) 
    dx, g,wstep = thick/n, int(nw/2) ,dw[1]-dw[0] #g is middle of the lorentzian
    nt = int(tf/dt) 

    #### Time steps 1,2,3 ####
    
    def funct(step):
        if step ==2: return [W2, N11, N21, N31]
        elif step==3: return [W3, N12, N22, N32]
        elif step==4: return[0,N13, N23,0]

    def funct2(step):
        if step ==1: return [N11, N21]
        elif step==2: return [N12, N22]
        elif step==3: return[N13, N23]

    def solve_int0(step):
        if step == 0:
            sol = np.zeros(nw)
            sol = [N0/((1+(dw[i]**2)*(t_F**2))) for i in range(nw)]
            return sol
        elif step == 1 or step==2 or step==3:
            g = funct2(step)
            N1,N2= g[0],g[1]
            sol, f= [], []
            for i in range(n):
                for j in range(nw): f.append((N1[i][j]-N2[i][j])/((1+(dw[j]**2)*(t_F**2))))
                sol.append(f)
                f =[]
            return sol

    def RK40(step):
        v = soln[step][0]*C3
        F1 = dx * (-I0*v)
        F2 = dx * (-(I0+F1/2)*v)
        F3 = dx * (-(I0+F2/2)*v)
        F4 = dx * (-(I0+F3)*v)
        sol = I0 + (F1 + 2*(F2+F3) + F4)/6
        return sol

    def PC0(step): 
        def f(y): return -y*C3*soln[step][0]
        if step==0 : y1 = RK4i
        elif step==1 : y1 = RK41
        elif step==2 : y1 = RK42
        elif step==3 : y1 = RK43
        yP,yC, f0 = np.zeros(n),np.zeros(n), f(I0)
        yC[0], yP[1], yC[1] = y1, y1 + (dx/2)*(3*f(y1) - f0),  y1 + (dx/2)*(f(yP[1]) - f(y1))
        for i in range(2,n):
            yP[i] = yC[i-1] + (dx/2)*(3*f(yC[i-1]) - f(yC[i-2]))
            yC[i] = yC[i-1] + (dx/2)*(f(yP[i]) + f(yC[i-1]))
        return yC[1:]

    def W0(step):
        if step ==1: Int = PCi
        elif step ==2: Int = PC1
        elif step ==3: Int = PC2
        W ,a= [],[]
        for i in range(n):
            for j in range(nw): a.append((C1*Int[i])/((dw[j]**2)*(t_F**2)+1+ (C1*Int[i]*t_F)))
            W.append(a)    
            a = []
        return W

    def int0(step):
        if step == 1: sol = sols1
        if step ==2: sol = sols2
        if step ==3: sol = sols3
        var,f = [],[]
        for i in range(n):
            for j in range(nw-1): f.append((sol[i][j+1]+sol[i][j])*(wstep/2))
            var.append(f)
            f = []
        var = np.array(var)
        return var

    def N2i(step):
        g = funct(step)
        if step==1:
            h ,c = [],[]
            for i in range(n):
                for j in range(nw): h.append(+W1[i][j]*N0*dt)
                c.append(h)
                h = []  
            return c
        elif step==2 or step==3:
            h ,c = [],[]
            W,N1,N2= g[0],g[1],g[2]
            for i in range(n):
                for j in range(nw): 
                    F1 = dt*(W[i][j]*N1[i][j] - (k_F+k_ISC+W[i][j])*N2[i][j])
                    F2 = dt*(W[i][j]*N1[i][j] - (k_F+k_ISC+W[i][j])*(N2[i][j]+F1/2))
                    F3 = dt*(W[i][j]*N1[i][j] - (k_F+k_ISC+W[i][j])*(N2[i][j]+F2/2))
                    F4 = dt*(W[i][j]*N1[i][j] - (k_F+k_ISC+W[i][j])*(N2[i][j]+F3))
                    h.append(N2[i][j] + (1/6)*(F1+2*F2+2*F3+F4))
                c.append(h)
                h = []  
            return c 

    def N3i(step):
        g = funct(step)
        if step == 1: return np.zeros((n,nw))
        elif step == 2:
            N2,N3= g[2],g[3]
            f ,c = [],[]
            for i in range(n):
                for j in range(nw): f.append(N2[i][j]*k_ISC*dt)
                c.append(f)
                f = []  
            return c
        elif step == 3:
            N2,N3= g[2],g[3]
            f ,c = [],[]
            for i in range(n):
                for j in range(nw):
                    F1 = dt* (N2[i][j]*k_ISC - k_31*N3[i][j])
                    F2 = dt* (N2[i][j]*k_ISC - k_31*(N3[i][j]+F1/2))
                    F3 = dt* (N2[i][j]*k_ISC - k_31*(N3[i][j]+F2/2))
                    F4 = dt* (N2[i][j]*k_ISC - k_31*(N3[i][j]+F3))
                    f.append(N3[i][j] + (1/6)*(F1+2*F2+2*F3+F4))
                c.append(f)
                f = [] 
            return c

    sols0 = solve_int0(0)
    sol0 = sum([(sols0[i+1]+sols0[i])*(wstep/2) for i in range(nw-1)])
    arr = np.empty(n)
    arr.fill(sol0)
    soln = [arr]
    RK4i = RK40(0)
    PCi = np.insert(PC0(0),0,I0)

    W1 = W0(1) 
    N21 = N2i(1)
    N31= N3i(1)
    N00,N11, func = np.zeros((n,nw)),[],[]
    N00.fill(N0)
    N00.tolist()
    for i in range(n):
        for j in range(nw): func.append(N00[i][j] - N21[i][j])
        N11.append(func)
        func = []
    sols1 = solve_int0(1)
    sol1 = int0(1).sum(axis = 1)
    soln.append(sol1)
    RK41 = RK40(1)
    PC1 = np.insert(PC0(1),0,I0)

    W2 = W0(2)  
    N22 = N2i(2)
    N32 = N3i(2)
    N00 = np.zeros((n,nw))
    N00.fill(N0)
    N00.tolist()
    N12, func2 = [],[]
    for i in range(n):
        for j in range(nw): func2.append(N00[i][j] - (N22[i][j]+N32[i][j]))
        N12.append(func2)
        func2 = []
    sols2 = solve_int0(2)
    sol2 = int0(2).sum(axis = 1)
    soln.append(sol2)
    RK42 = RK40(2)
    PC2 = np.insert(PC0(2),0,I0)

    W3 = W0(3)
    N23 = N2i(3)
    N33 = N3i(3)
    N13, func3 = [],[]
    for i in range(n):
        for j in range(nw): func3.append(N00[i][j] - (N23[i][j]+N33[i][j]))
        N13.append(func3)
        func3 = []
    sols3 = solve_int0(3)
    sol3 = int0(3).sum(axis = 1)
    soln.append(sol3)
    RK43 = RK40(3)
    PC3 = np.insert(PC0(3),0,I0)
    
    #### Time Steps 4-nt ####

    Nin1, Nin2, Nin3 = np.empty((n,nw)),np.zeros((n,nw)),np.zeros((n,nw))
    Nin1.fill(N0)
    sol00 = np.empty(n)
    sol00.fill(sol0)
    N1_pop, N2_pop, N3_pop = np.append([Nin1], [N11,N12,N13], axis=0), np.append([Nin2], [N21,N22,N23], axis=0),np.append([Nin3], [Nin3,N32,N33], axis=0)
    Intens_PC ,alpha = np.append([PCi], [PC1,PC2,PC3], axis=0),np.append([sol00], [sol1, sol2, sol3], axis = 0)

    for k in range(4,nt):
        #if Intens_PC < I0:
        def W():
            a,W = [],[]
            for i in range(n):
                for j in range(nw): a.append((C1*Intens_PC[k-1][i])/((dw[j]**2)*(t_F**2)+1+C1*Intens_PC[k-1][i]*t_F))
                W.append(a)    
                a = []
            return W

        def N2():
            f ,c = [],[]
            for i in range(n):
                for j in range(nw):
                    F1 = dt*(W[i][j]*N1_pop[k-1][i][j] - (k_F+k_ISC+W[i][j])*N2_pop[k-1][i][j])
                    F2 = dt*(W[i][j]*N1_pop[k-1][i][j] - (k_F+k_ISC+W[i][j])*(N2_pop[k-1][i][j]+F1/2))
                    F3 = dt* (W[i][j]*N1_pop[k-1][i][j] - (k_F+k_ISC+W[i][j])*(N2_pop[k-1][i][j]+F2/2))
                    F4 =  dt* (W[i][j]*N1_pop[k-1][i][j] - (k_F+k_ISC+W[i][j])*(N2_pop[k-1][i][j]+F3))
                    f.append(N2_pop[k-1][i][j] + (1/6)*(F1+2*F2+2*F3+F4))
                c.append(f)
                f = [] 
            return c

        def N3():
            f ,c = [],[]
            for i in range(n):
                for j in range(nw): 
                    F1 = dt*(N2_pop[k-1][i][j]*k_ISC - k_31*N3_pop[k-1][i][j])
                    F2 = dt*(N2_pop[k-1][i][j]*k_ISC - k_31*(N3_pop[k-1][i][j]+F1/2))
                    F3 = dt* (N2_pop[k-1][i][j]*k_ISC - k_31*(N3_pop[k-1][i][j]+F2/2))
                    F4 =  dt* (N2_pop[k-1][i][j]*k_ISC - k_31*(N3_pop[k-1][i][j]+F3))
                    f.append(N3_pop[k-1][i][j] + (1/6)*(F1+2*F2+2*F3+F4))
                c.append(f)
                f = [] 
            return c

        def solve_int():
            sol, f= [], []
            for i in range(n):
                for j in range(nw): f.append((abs(N1[i][j])-abs(N2[i][j]))/(1+(dw[j]**2)*(t_F**2)))
                sol.append(f)
                f =[]
            return sol

        def int0():
            var,f = [],[]
            for i in range(n):
                for j in range(nw-1): f.append((sols[i][j+1]+sols[i][j])*(wstep/2))
                var.append(f)
                f = []
            var = np.array(var)
            return var

        def RK4(sol): 
            v = C3*sol[0]
            F1 = dx * (-I0*v)
            F2 = dx * (-(I0+F1/2)*v) 
            F3 = dx * (-(I0+F2/2)*v)
            F4 = dx * (-(I0+F3)*v)
            sol = I0 + (F1 + 2*(F2+F3) + F4)/6
            return sol

        def PC(): #Predictor Corrector Method 
            def f(y): return -y*C3*sol[0]
            yP,yC, f0 , y1= np.zeros(n),np.zeros(n), -I0*C3*sol[0] , RK4
            yC[0], yP[1], yC[1] = y1, y1 + (dx/2)*(3*f(y1) - f0),  y1 + (dx/2)*(f(yP[1]) - f(y1))
            for j in range(2,n):
                yP[j] = yC[j-1] + (dx/2)*(3*f(yC[j-1]) - f(yC[j-2]))
                yC[j] = yC[j-1] + (dx/2)*(f(yP[j]) + f(yC[j-1]))
            return yC[1:]

        W = W()
        N2 = N2()
        N3 = N3()
        N1, func1 = [],[]
        for i in range(n):
            for j in range(nw): func1.append(N00[i][j] - (N2[i][j]+N3[i][j]))
            N1.append(func1)
            func1 = []
        sols = solve_int()
        sol = int0().sum(axis = 1)
        RK4 = RK4(sol)
        PC = np.insert(PC(),0,I0)
        Intens_PC = np.append(Intens_PC,[PC], axis = 0)
        alpha = np.append(alpha, [sol], axis = 0)
        N1_pop,N2_pop,N3_pop = np.append(N1_pop,[N1], axis = 0),np.append(N2_pop,[N2], axis = 0),np.append(N3_pop,[N3], axis = 0)
    
    #### Check Population Values ####
    for i,j,k in zip(range(nt), range(n), range(nw)):
        if N1_pop[i][j][k] < 0: raise MyError('Negative Population Error: dt is too large')
        elif N2_pop[i][j][k] < 0: raise MyError('Negative Population Error: dt is too large')
        elif N3_pop[i][j][k] < 0: raise MyError('Negative Population Error: dt is too large')
            
    k = int(n-1)
    ans_PC = np.zeros(nt)
    for i in range(nt): ans_PC[i] = Intens_PC[i,k]
    t = np.linspace(0, tf, nt)
    x = np.linspace(0,thick,n)
    j = int(nt-1)

    ######## FIGURES ########
    
    plt.rcParams["figure.figsize"] = [7.50, 3.50]
    plt.rcParams["figure.autolayout"] = True

    fig0 = plt.figure(figsize=(12,10))
    txt1 = 'Concentration(ppm):', conc, 'Thickness(m):',thick
    txt2 = 'Beam Diameter(m):', beam_diam, 'Input Power:', P_in
    txt3 = 'Incident Intensity:', I0, '# Space Steps', n, 
    txt4 = 'Size of time step:', dt, 'Duration of Simulation:', tf, 
    txt5 = 'wlim:', wl*k_F, 'Number of Frequency steps:', nw 
    txt6 = '# Time Steps:', nt
    plt.text(0.05,0.95,txt1, size=15)
    plt.text(0.05,0.8,txt2, size=15)
    plt.text(0.05,0.6,txt3, size=15)
    plt.text(0.05,0.4,txt4, size=15)
    plt.text(0.05,0.2,txt5, size=15)
    plt.text(0.05,0.05,txt6, size=15)
    
    
    fig1, axis = plt.subplots(1, 3, figsize = (20,4))
    axis[0].plot(t, ans_PC, color='r')
    axis[0].set_title("Transmitted Intensity vs. Time")

    axis[1].plot(x, Intens_PC[j])
    axis[1].set_title("Transmitted Intensity vs. Space")

    for i in range(0,j,30): axis[2].plot(x,Intens_PC[i])
    axis[2].set_title("I vs.x over time")

    fig2 = plt.figure()
    plt.plot(t, [alpha[i][0] for i in range(nt)], color='purple') #at start of crystal
    plt.plot(t, [alpha[i][k] for i in range(nt)], color='orange') #at end of crystal
    plt.xlabel('time (s)')
    plt.ylabel('Absorption Coefficient')
    
    fig3, axis = plt.subplots(3, 3, figsize = (20,10))

    f1 = [N1_pop[i][k][g] for i in range(nt)]
    f2 = [N2_pop[i][k][g] for i in range(nt)]
    f3 = [N3_pop[i][k][g] for i in range(nt)]
    g1 = [N1_pop[i][0][g] for i in range(nt)]
    g2 = [N2_pop[i][0][g] for i in range(nt)]
    g3 = [N3_pop[i][0][g] for i in range(nt)]

    axis[0, 0].plot(t,f1) 
    axis[0, 0].set_title("N1 over time (at the back of the crystal)")

    axis[0, 1].plot(t,f2) 
    axis[0, 1].set_title("N2 over time (at the back of the crystal)")

    axis[0, 2].plot(t,f3) 
    axis[0, 2].set_title("N3 over time (at the back of the crystal)")

    axis[1, 0].plot(t,g1) 
    axis[1, 0].set_title("N1 over time (at the front of the crystal)")

    axis[1, 1].plot(t,g2) 
    axis[1, 1].set_title("N2 over time (at the front of the crystal)")

    axis[1, 2].plot(t,g3) 
    axis[1, 2].set_title("N3 over time (at the front of the crystal)")

    axis[2, 0].plot(t,g1) 
    axis[2, 0].set_title("N1 over time (at the center of the crystal)")

    axis[2, 1].plot(t,g2) 
    axis[2, 1].set_title("N2 over time (at the center of the crystal)")

    axis[2, 2].plot(t,g3) 
    axis[2, 2].set_title("N3 over time (at the center of the crystal)")

    fig4, axis = plt.subplots(1, 3, figsize = (15,4))

    f1 = [N1_pop[j][i][g] for i in range(n)]
    f2 = [N2_pop[j][i][g] for i in range(n)]
    f3 = [N3_pop[j][i][g] for i in range(n)]

    axis[0].plot(x,f1) 
    axis[0].set_title("N1 vs. x (at the last time step)", y=1.08)

    axis[1].plot(x,f2) 
    axis[1].set_title("N2 vs. x (at the last time step)", y=1.08)

    axis[2].plot(x,f3) 
    axis[2].set_title("N3 vs. x (at the last time step)", y=1.08)

    fig5, axis = plt.subplots(2, 3, figsize = (20,9))

    st = 50
    sx = 5

    f1,f2,f3,g1,g2,g3 =[],[],[],[],[],[]
    h1, h2, h3, m1, m2, m3 = [],[],[],[],[],[]
    #display(len(dw))

    for p in range(1,nt,st):
        for i in range(nw):
            h1.append(N1_pop[p][k][i])
        f1.append(h1)
        h1 = []

    for p in range(1,nt,st):
        for i in range(nw):
            h2.append(N2_pop[p][k][i])
        f2.append(h2)
        h2 = []
    for p in range(1,nt,st):
        for i in range(nw):
            h3.append(N3_pop[p][k][i])
        f3.append(h3)
        h3 = []
    for p in range(0,n,sx):
        for i in range(nw):
            m1.append(N1_pop[j][p][i])
        g1.append(m1)
        m1 = []
    for p in range(0,n,sx):
        for i in range(nw):
            m2.append(N2_pop[j][p][i])
        g2.append(m2)
        m2 = []
    for p in range(0,n,sx):
        for i in range(nw):
            m3.append(N3_pop[j][p][i])
        g3.append(m3)
        m3 = []

    var1 = int(nt/st)
    var2 = int(n/sx)

    for i in range(var1): axis[0, 0].plot(dw,f1[i]) 
    axis[0, 0].set_title("N1 vs. dw as a function of t")

    for i in range(var1): axis[0, 1].plot(dw,f2[i]) 
    axis[0, 1].set_title("N2 vs. dw as a function of t")

    for i in range(var1): axis[0,2].plot(dw,f3[i]) 
    axis[0, 2].set_title("N3 vs. dw as a function of t") 

    for i in range(var2): axis[1, 0].plot(dw,g1[i]) 
    axis[1, 0].set_title("N1 vs. dw as a function of x ")

    for i in range(var2): axis[1, 1].plot(dw,g2[i]) 
    axis[1, 1].set_title("N2 vs. dw as a function of x ")

    for i in range(var2): axis[1, 2].plot(dw,g3[i]) 
    axis[1, 2].set_title("N3 vs. dw as a function of x ")
    
    #### Save PDF ####
    
    def save_multi_image(filename):
        pp = PdfPages(filename)
        fig_nums = plt.get_fignums()
        figs = [plt.figure(n) for n in fig_nums]
        for fig in figs:
            fig.savefig(pp, format='pdf')
        pp.close()
    
    save_multi_image(filename)
    
    return Intens_PC, N1_pop, N2_pop, N3_pop, alpha


# Results

In [ ]:
Intensity,N1,N2,N3,alpha = Model(1* 10**(-3),0.1,3 * 10**(-3),0,1.75*10**4,40,9*10**(-9),1*10**(-6),20,100,"attempt1.pdf")

# Interactive Graphs

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual,Box
import ipywidgets as widgets

In [ ]:
#Intensity and Absorption Coefficient
IPC = Intensity
a1 = alpha

nw = 100
g = int(nw/2)
n = 40
nt = 11111
thick = 10**(-3)
tf = 100*10**(-6)
t = np.linspace(0, tf, nt)
x = np.linspace(0,thick,n)

def I_vs_x(t):
    y = IPC[t]
    plt.plot(x, y)
    plt.ylabel('Intensity')
    plt.xlabel('x')
    plt.title('Intensity vs. x at different t')
    return plt.figure()

def I_vs_t(x):
    f = []
    for i in range(nt): f.append(IPC[i,x])
    plt.plot(t, f)
    plt.ylabel('Intensity (W/m^2)')
    plt.xlabel('time (s)')
    plt.title('Intensity vs. t at different x')
    return plt.figure()

def alpha_vs_t(x):
    f = []
    for i in range(nt): f.append(a1[i,x])
    plt.plot(t, f)
    plt.ylabel('Absorption Coefficient ')
    plt.xlabel('time (s)')
    plt.title('alpha vs. t at different x')
    return plt.figure()

def alpha_vs_x(t):
    f = []
    for i in range(n): f.append(a1[t,i])
    plt.plot(x, f)
    plt.ylabel('Absorption Coefficient ')
    plt.xlabel('time (s)')
    plt.title('alpha vs. t at different x')
    return plt.figure()

w1 = interactive(I_vs_x,t = widgets.IntSlider(min=0, max=nt-1, step=50, value=0))
w2 = interactive(I_vs_t, x = widgets.IntSlider(min=0, max=n-1, step=1, value=0))
w3 = interactive(alpha_vs_t, x = widgets.IntSlider(min=0, max=n-1, step=1, value=0) )
w4 = interactive(alpha_vs_x, t = widgets.IntSlider(min=0, max=nt-1, step=50, value=0))

items = [w1, w2, w3,w4]
box = Box(children=items)
box

In [ ]:
#Populations 

Npop = N1 #change N1 N2 N3 

k_F = 4.2 * 10**7 
wlim = 20*k_F 
nw = 100 
dw = np.linspace(-wlim, wlim, nw)
st = 50
sx = 5

def N_vs_dwx(x):
    for p in range(1,nt,st):
        f1,h1 =[],[]
        for i in range(nw):
            h1.append(Npop[p,x,i])
        f1.append(h1)
        h1 = []
    plt.plot(dw, f1[0])
    plt.ylabel('Ni_pop ')
    plt.xlabel('frequency (1/s)')
    plt.title('Population vs. frequency at different x')
    return plt.figure()

def N_vs_dwt(t):
    for p in range(1,n,sx):
        f1,h1 =[],[]
        for i in range(nw):
            h1.append(Npop[t,p,i]) 
        f1.append(h1)
        h1 = []
    plt.plot(dw, f1[0])
    plt.ylabel('Ni_pop ')
    plt.xlabel('frequency (1/s)')
    plt.title('Population vs. frequency at different t')
    return plt.figure()

def N_vs_tx(x):
    f = []
    for i in range(nt): f.append(Npop[i,x,g])
    plt.plot(t, f)
    plt.ylabel('Ni_pop ')
    plt.xlabel('time (s)')
    plt.title('Population vs. time at different x')
    return plt.figure()

def N_vs_xt(t):
    f = []
    for i in range(n): f.append(Npop[t,i,g])
    plt.plot(x, f)
    plt.ylabel('Ni_pop ')
    plt.xlabel('x (m)')
    plt.title('Population vs. space at different t')
    return plt.figure()

w1 = interactive(N_vs_dwx,x =widgets.IntSlider(min=0, max=n-1, step=1, value=0))
w2 = interactive(N_vs_dwt, t = widgets.IntSlider(min=0, max=nt-1, step=30, value=0))
w3 = interactive(N_vs_tx,x = widgets.IntSlider(min=0, max=n-1, step=1, value=0))
w4 = interactive(N_vs_xt, t = widgets.IntSlider(min=0, max=nt-1, step=10, value=0))
items = [w1, w2,w3,w4]
box = Box(children=items)
box